In [1]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPool2D
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
x=[]
path = "../data/all_photos"'
for folder in os.listdir(path):
    sub_path=path+"/"+folder
    for img in os.listdir(sub_path):
        image_path=sub_path+"/"+img
        img_arr=cv2.imread(image_path)
        img_arr=cv2.resize(img_arr,(224,224))
        x.append(img_arr)
X = np.array(x)

In [3]:
datagen = ImageDataGenerator(rescale = 1./255)
dataset = datagen.flow_from_directory(path,
                                                 target_size = (224, 224),
                                                 batch_size = 1,
                                                 class_mode = 'sparse')
Y=dataset.classes

Found 830 images belonging to 5 classes.


In [16]:
conf_matrix = np.zeros((5,5))

In [17]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)
# define 5-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
cvscores = []
for train, test in kfold.split(X, Y):

    #do not train the pre-trained layers of ResNet50
    model = Sequential()
    model.add(Conv2D(16,2,padding="valid", activation="relu", input_shape=(224,224,3), kernel_regularizer=regularizers.l1(0.02)))
    model.add(MaxPool2D())
    #model.add(Dropout(0.7))


    model.add(Conv2D(16, 2, padding="valid", activation="relu"))
    model.add(MaxPool2D())


    model.add(Conv2D(32, 1, padding="valid", activation="relu"))
    model.add(MaxPool2D())
    model.add(Dropout(0.7))

    model.add(Flatten())
    model.add(Dense(128,activation="relu", kernel_regularizer=regularizers.l1(0.002)))
    model.add(Dense(5, activation="softmax"))
    # Compile model
    optimizer = keras.optimizers.Nadam(learning_rate=0.001)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
    # Fit the model
    model.fit(X[train], Y[train], epochs=15, batch_size=16, verbose=1, callbacks=[early_stopping])
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    y_pred=model.predict(X[test])
    y_pred=np.argmax(y_pred,axis=1)
    conf_matrix += confusion_matrix(y_pred, Y[test])
 
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Epoch 1/15
47/47 [==============================] - 1s 15ms/step - loss: 16.9712 - accuracy: 0.8902
Epoch 2/15
47/47 [==============================] - 1s 13ms/step - loss: 5.8866 - accuracy: 0.9572
Epoch 3/15
47/47 [==============================] - 1s 13ms/step - loss: 4.1961 - accuracy: 0.9679
Epoch 4/15
47/47 [==============================] - 1s 12ms/step - loss: 3.7009 - accuracy: 0.9545
Epoch 5/15
47/47 [==============================] - 1s 12ms/step - loss: 3.0888 - accuracy: 0.9786
Epoch 6/15
47/47 [==============================] - 1s 12ms/step - loss: 3.1852 - accuracy: 0.9612
Epoch 7/15
47/47 [==============================] - 1s 12ms/step - loss: 2.9112 - accuracy: 0.9598
Epoch 8/15
47/47 [==============================] - 1s 12ms/step - loss: 2.4988 - accuracy: 0.9692
Epoch 9/15
47/47 [==============================] - 1s 12ms/step - loss: 2.4548 - accuracy: 0.9692
Epoch 10/15
47/47 [==============================] - 1s 13ms/step - loss: 2.3516 - accuracy: 0.9772
Epoch 11

47/47 [==============================] - 1s 13ms/step - loss: 2.5209 - accuracy: 0.9759
Epoch 9/15
47/47 [==============================] - 1s 13ms/step - loss: 3.0568 - accuracy: 0.9732
Epoch 10/15
47/47 [==============================] - 1s 13ms/step - loss: 2.7695 - accuracy: 0.9665
Epoch 11/15
47/47 [==============================] - 1s 13ms/step - loss: 2.3271 - accuracy: 0.9826
Epoch 12/15
47/47 [==============================] - 1s 13ms/step - loss: 2.4047 - accuracy: 0.9652
Epoch 13/15
47/47 [==============================] - 1s 13ms/step - loss: 2.3137 - accuracy: 0.9786
Epoch 14/15
47/47 [==============================] - 1s 13ms/step - loss: 2.3771 - accuracy: 0.9853
Epoch 15/15
47/47 [==============================] - 1s 13ms/step - loss: 2.2054 - accuracy: 0.9813
accuracy: 100.00%
Epoch 1/15
47/47 [==============================] - 1s 13ms/step - loss: 19.1127 - accuracy: 0.8594
Epoch 2/15
47/47 [==============================] - 1s 13ms/step - loss: 7.9670 - accuracy: 0.9

47/47 [==============================] - 1s 13ms/step - loss: 4.1130 - accuracy: 0.9705
Epoch 4/15
47/47 [==============================] - 1s 13ms/step - loss: 3.4802 - accuracy: 0.9705
Epoch 5/15
47/47 [==============================] - 1s 13ms/step - loss: 3.5379 - accuracy: 0.9639
Epoch 6/15
47/47 [==============================] - 1s 13ms/step - loss: 3.1446 - accuracy: 0.9719
Epoch 7/15
47/47 [==============================] - 1s 13ms/step - loss: 2.8695 - accuracy: 0.9598
Epoch 8/15
47/47 [==============================] - 1s 13ms/step - loss: 2.7240 - accuracy: 0.9853
Epoch 9/15
47/47 [==============================] - 1s 13ms/step - loss: 2.6206 - accuracy: 0.9746
Epoch 10/15
47/47 [==============================] - 1s 13ms/step - loss: 2.6009 - accuracy: 0.9652
Epoch 11/15
47/47 [==============================] - 1s 13ms/step - loss: 2.3954 - accuracy: 0.9839
Epoch 12/15
47/47 [==============================] - 1s 13ms/step - loss: 2.4306 - accuracy: 0.9692
Epoch 13/15
47/47 

Epoch 1/15
47/47 [==============================] - 1s 13ms/step - loss: 15.1697 - accuracy: 0.9036
Epoch 2/15
47/47 [==============================] - 1s 13ms/step - loss: 4.6376 - accuracy: 0.9572
Epoch 3/15
47/47 [==============================] - 1s 13ms/step - loss: 3.6793 - accuracy: 0.9558
Epoch 4/15
47/47 [==============================] - 1s 13ms/step - loss: 3.3202 - accuracy: 0.9679
Epoch 5/15
47/47 [==============================] - 1s 13ms/step - loss: 2.9945 - accuracy: 0.9746
Epoch 6/15
47/47 [==============================] - 1s 13ms/step - loss: 2.7543 - accuracy: 0.9732
Epoch 7/15
47/47 [==============================] - 1s 13ms/step - loss: 2.6297 - accuracy: 0.9746
Epoch 8/15
47/47 [==============================] - 1s 13ms/step - loss: 2.8450 - accuracy: 0.9719
Epoch 9/15
47/47 [==============================] - 1s 13ms/step - loss: 2.5811 - accuracy: 0.9692
Epoch 10/15
47/47 [==============================] - 1s 13ms/step - loss: 2.5170 - accuracy: 0.9813
Epoch 11

In [18]:
conf_matrix

array([[162.,   1.,   1.,   0.,   0.],
       [  0., 164.,   0.,   1.,   0.],
       [  4.,   0., 165.,   0.,   0.],
       [  0.,   1.,   0., 165.,   0.],
       [  0.,   0.,   0.,   0., 166.]])

In [22]:
model.save('Custom NN K-fold.keras')

In [21]:
example=cv2.imread('relax.png')
example=cv2.resize(example,(224,224))
example = np.expand_dims(example, axis=0)
np.argmax(model.predict(example))
model.predict(example)

array([[3.9750226e-03, 9.7178627e-04, 1.6083720e-03, 1.3172219e-03,
        9.9212754e-01]], dtype=float32)